In [1]:
# To allow importing of packages in two directories up
import sys
sys.path.append("../..")

# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.simplefilter('ignore')

from sklearn import datasets
from tqdm.auto import tqdm
import scipy.stats as sc

from core.wca import wca_mean
from experiments.dataset_file.reading_file import read_abalone
np.random.seed(21)

In [2]:
# Setting the number of clusters
k=2

# Number of columns to show
col_num = 4

### Random sampling

In [ ]:
#X, y, df= get_data(data)
X, y, df= read_abalone()

C_random = dict()
for cluster in range(k):
    C_random[cluster] = pd.DataFrame(X[:1])

# Initializing the pv-alues
pval_random = dict()
for cluster in range(k):
    pval_random[cluster] = dict()
    for i in C_random[0].columns:
        pval_random[cluster][i] = list()

for i in tqdm(range(int(len(X)/k)), desc = "Iterations: "):
    for cluster in range(k):
        idx = np.random.randint(low=0,high=len(X))
        C_random[cluster] = C_random[cluster].append(X.iloc[idx])
        X.drop(idx, inplace=True)
        X.reset_index(drop=True,inplace=True)
        C_random[cluster].reset_index(drop=True,inplace=True)
        for i in C_random[0].columns:
            pval_random[cluster][i].append(sc.ks_2samp(C_random[cluster][i],df.drop('target',axis=1)[i])[1])

### WCA

In [4]:
#X, y, df= get_data(data)
X, y, df= read_abalone()

# Running the wca_mean function
C, pval = wca_mean(X, k, df)

KeyboardInterrupt: 

### Plotting the result

In [ ]:
fig, axes = plt.subplots(k, len(C[0].columns[:col_num]), figsize=(15,k*2.5))

# Plotting the KS p-value
for cluster in range(k):
    for i,j in enumerate(C[0].columns[:col_num]):
        axes[cluster, i].plot(pval_random[cluster][j], marker='.',color="Blue")
        axes[cluster, i].plot(pval[cluster][j], marker='+',color='Black')
        axes[cluster, i].legend(['Random','WCA'])
        

# Setting columns headers
for ax, col in zip(axes[0], C[0].columns[:col_num]):
    ax.set_title(col)

# Setting row titles
for ax, row in zip(axes[:,0], range(k)):
    ax.set_ylabel("Cluster {} ".format(row), rotation=90, size='large')
        
plt.tight_layout()
plt.show()      